# Compare H151 treated 5xFAD v.s. DMSO treated 5xFAD

In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(SeuratWrappers)
    library(patchwork)
    library(ggplot2)
    library(repr)
    library(gridExtra)
    library(edgeR)
    library(SingleCellExperiment)
    library(Matrix)
    library(scran)
    library(tidyverse)
    library(ggrepel)
    library(scater)
    library(fs)
    library(Signac)
})
options(future.globals.maxSize = 1e9)
options(Seurat.object.assay.version = "v5")
options(ggrepel.max.overlaps = Inf)

In [2]:
setwd("~/5XFAD_mouse/data/")

In [3]:
seurat_object <- readRDS(
    "mouseAD_H151_ATAC_signac_object.rds")

In [4]:
unique(seurat_object$orig.ident)

[1] "2528" "2529" "2530" "2714" "2816" "2891" "2893" "2894" "3000" "3001"
[11] "3002" "3003" "3501" "3502" "3503"

In [5]:
unique(seurat_object$final_cluster)

[1] "Oligo"          "DG"             "SUB_2"          "CA3"           
 [5] "Microglia"      "SUB-ProS"       "Sst"            "OPC"           
 [9] "CA1"            "Pvalb"          "Astro"          "Vip"           
[13] "SUB_1"          "Meis2"          "Lamp5"          "SUB_3"         
[17] "CA2"            "Sncg"           "Endo"           "Chandelier"    
[21] "IOL"            "PVM"            "Choroid-plexus" "VLMC"          
[25] "SMC-Peri"

In [6]:
unique(seurat_object$genotype)

[1] "5XFAD" "WT"

In [7]:
table(seurat_object$genotype, seurat_object$orig.ident)

       
         2528  2529  2530  2714  2816  2891  2893  2894  3000  3001  3002  3003
  5XFAD 10579  9108  5624  6276  2370  9576 10793  8450     0     0     0     0
  WT        0     0     0     0     0     0     0     0  8256  7817 10103  6336
       
         3501  3502  3503
  5XFAD     0     0     0
  WT     9190  7222  9222

In [8]:
table(seurat_object$genotype, seurat_object$intervention)

       
         DMSO  H151
  5XFAD 34443 28333
  WT    25634 32512

In [9]:
table(seurat_object$orig.ident, seurat_object$intervention)

      
        DMSO  H151
  2528     0 10579
  2529     0  9108
  2530  5624     0
  2714     0  6276
  2816     0  2370
  2891  9576     0
  2893 10793     0
  2894  8450     0
  3000     0  8256
  3001     0  7817
  3002     0 10103
  3003     0  6336
  3501  9190     0
  3502  7222     0
  3503  9222     0

In [10]:
seurat_object_wt <- subset(seurat_object, subset = genotype == "WT")

## Start processing

In [11]:
subclasses <- path_ext_remove(path_file(dir_ls("H151_raw_peaks/")))

In [12]:
table(seurat_object$final_cluster)


         Astro            CA1            CA2            CA3     Chandelier 
           364          16213            996           7766            822 
Choroid-plexus             DG           Endo            IOL          Lamp5 
            47          27171            113            186           1697 
         Meis2      Microglia          Oligo            OPC          Pvalb 
          3976           6922          25842           2167           1341 
           PVM       SMC-Peri           Sncg            Sst          SUB_1 
           142             53            972           2114           3349 
         SUB_2          SUB_3       SUB-ProS            Vip           VLMC 
         12665           2020           2691           1161            132 

In [14]:
table(seurat_object_wt$final_cluster)


         Astro            CA1            CA2            CA3     Chandelier 
           234           8810            508           3894            414 
Choroid-plexus             DG           Endo            IOL          Lamp5 
            31          15511             28             54            819 
         Meis2      Microglia          Oligo            OPC          Pvalb 
          2393           1619          12446           1024            545 
           PVM       SMC-Peri           Sncg            Sst          SUB_1 
            77             38            481            925           1830 
         SUB_2          SUB_3       SUB-ProS            Vip           VLMC 
          4088            940            879            494             64 

In [16]:
for (subclass in unique(seurat_object_wt$final_cluster)) {
    if (subclass %in% c(
        "Choroid-plexus", "Endo", "IOL", 
        "PVM", "SMC-Peri", "VLMC", "DAM"
    )) {
        next
    }
    
    bed_file <- sprintf("H151_raw_peaks/%s.bed", subclass)
    bed_table <- read.table(bed_file, sep = "\t", header = FALSE)
    colnames(bed_table) <- c("chrom", "start", "end", "label")
    kept_peaks <- paste(bed_table$chrom, bed_table$start, bed_table$end, sep = "-")
    curr_object <- subset(seurat_object_wt, subset = final_cluster == subclass)
    curr_object <- curr_object[kept_peaks]
    
    curr_sce <- SingleCellExperiment(
        list(counts = LayerData(curr_object, layer = "counts")),
        colData = curr_object@meta.data
    )
    curr_sce_aggr <- aggregateAcrossCells(curr_sce, curr_object$orig.ident)
    
    curr_counts <- counts(curr_sce_aggr)
    Group <- curr_sce_aggr$intervention
    Group <- factor(Group, levels = c("DMSO", "H151"))
    
    curr_dge <- DGEList(counts = curr_counts, group = Group, remove.zeros = TRUE)
    # keep <- filterByExpr(curr_dge, min.count = 2, min.prop = 0.5)
    # curr_dge <- curr_dge[keep, , keep.lib.sizes=FALSE]
    curr_dge <- calcNormFactors(curr_dge, method = "TMM")
    
    curr_design <- model.matrix(~ 0 + Group)
    colnames(curr_design) <- levels(Group)
    curr_dge <- estimateDisp(curr_dge, curr_design, robust = TRUE)
    curr_fit <- glmQLFit(curr_dge, curr_design)
    curr_glf <- glmQLFTest(curr_fit, contrast = c(-1, 1))
    
    write.table(
        topTags(curr_glf, n = Inf)$table, 
        file.path("DAR_H151_WT/", paste0(subclass, ".result.tsv")), 
        row.names = TRUE,
        col.names = TRUE,
        quote = FALSE,
        sep = "\t"
    )
}

Removing 8 rows with all zero counts

Removing 20 rows with all zero counts

Removing 10215 rows with all zero counts

Removing 910 rows with all zero counts

Removing 1796 rows with all zero counts

Removing 424 rows with all zero counts

Removing 2749 rows with all zero counts

Removing 10036 rows with all zero counts

Removing 10109 rows with all zero counts

Removing 4579 rows with all zero counts

Removing 15947 rows with all zero counts

Warning message:
“21 very small variances detected, have been offset away from zero”
Warning message:
“Zero sample variances detected, have been offset away from zero”
Removing 15657 rows with all zero counts

Removing 14916 rows with all zero counts

Removing 12438 rows with all zero counts

Removing 7476 rows with all zero counts

Removing 15629 rows with all zero counts

Removing 14770 rows with all zero counts

Warning message:
“18 very small variances detected, have been offset away from zero”
Warning message:
“Zero sample variances detected